<a href="https://colab.research.google.com/github/talha-0/MedicalMind/blob/main/Medical_Mind_For_Doctors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mimic III Dataset

In [ ]:
!pip install -qU pinecone-client langchain openai sqlalchemy tiktoken cohere ipython-sql

In [ ]:
%load_ext sql

In [ ]:
!wget -r -N -c -np https://physionet.org/files/mimiciii-demo/1.4/

--2024-01-22 06:11:45--  https://physionet.org/files/mimiciii-demo/1.4/
Resolving physionet.org (physionet.org)... 18.18.42.54
Connecting to physionet.org (physionet.org)|18.18.42.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘physionet.org/files/mimiciii-demo/1.4/index.html’

physionet.org/files     [ <=>                ]   3.50K  --.-KB/s    in 0s      

Last-modified header missing -- time-stamps turned off.
2024-01-22 06:11:46 (374 MB/s) - ‘physionet.org/files/mimiciii-demo/1.4/index.html’ saved [3582]

Loading robots.txt; please ignore errors.
--2024-01-22 06:11:46--  https://physionet.org/robots.txt
Reusing existing connection to physionet.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 22 [text/plain]
Saving to: ‘physionet.org/robots.txt’

physionet.org/robot 100%[===================>]      22  --.-KB/s    in 0s      

2024-01-22 06:11:47 (6.23 MB/s) - ‘physionet.org/robots.txt’ saved [22/22]

--20

In [ ]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, text
import pandas as pd
import os

# Create a SQLAlchemy engine with an in-memory SQLite database
engine = create_engine("sqlite:///mimic3.db")
engine.connect()

In [ ]:
csv_directory = '/content/physionet.org/files/mimiciii-demo/1.4'
# List to store table objects
tables = []

# Iterate over CSV files in the directory
for file_name in os.listdir(csv_directory):
  if file_name.endswith(".csv"):
    # Use the CSV file name (without extension) as the table name
    table_name = os.path.splitext(file_name)[0]
    tables.append(table_name)
    # Read CSV file into a pandas DataFrame
    df = pd.read_csv(os.path.join(csv_directory, file_name), low_memory=False)
    print(df.head())

    # Insert data into the table
    df.to_sql(con=engine,name = table_name,  if_exists='replace', index=False)

   row_id  subject_id gender                  dob                  dod  \
0    9467       10006      F  2094-03-05 00:00:00  2165-08-12 00:00:00   
1    9472       10011      F  2090-06-05 00:00:00  2126-08-28 00:00:00   
2    9474       10013      F  2038-09-03 00:00:00  2125-10-07 00:00:00   
3    9478       10017      F  2075-09-21 00:00:00  2152-09-12 00:00:00   
4    9479       10019      M  2114-06-20 00:00:00  2163-05-15 00:00:00   

              dod_hosp              dod_ssn  expire_flag  
0  2165-08-12 00:00:00  2165-08-12 00:00:00            1  
1  2126-08-28 00:00:00                  NaN            1  
2  2125-10-07 00:00:00  2125-10-07 00:00:00            1  
3                  NaN  2152-09-12 00:00:00            1  
4  2163-05-15 00:00:00  2163-05-15 00:00:00            1  
   row_id  subject_id  hadm_id  seq_num  icd9_code
0    3994       10114   167957        1       3605
1    3995       10114   167957        2       3722
2    3996       10114   167957        3       88

In [ ]:
tables

['PATIENTS',
 'PROCEDURES_ICD',
 'D_ICD_PROCEDURES',
 'ICUSTAYS',
 'NOTEEVENTS',
 'DATETIMEEVENTS',
 'MICROBIOLOGYEVENTS',
 'CALLOUT',
 'D_ICD_DIAGNOSES',
 'TRANSFERS',
 'DIAGNOSES_ICD',
 'ADMISSIONS',
 'DRGCODES',
 'D_CPT',
 'CHARTEVENTS',
 'OUTPUTEVENTS',
 'PRESCRIPTIONS',
 'D_LABITEMS',
 'INPUTEVENTS_MV',
 'INPUTEVENTS_CV',
 'CAREGIVERS',
 'SERVICES',
 'PROCEDUREEVENTS_MV',
 'LABEVENTS',
 'D_ITEMS',
 'CPTEVENTS']

In [ ]:
# Confirm data insertion
for table_name in tables:
  result_data = pd.read_sql(text(f"SELECT * FROM '{table_name}' LIMIT 5"),engine.connect())
  print(f"\nContents of table '{table_name}':")
  print(result_data)


Contents of table 'PATIENTS':
   row_id  subject_id gender                  dob                  dod  \
0    9467       10006      F  2094-03-05 00:00:00  2165-08-12 00:00:00   
1    9472       10011      F  2090-06-05 00:00:00  2126-08-28 00:00:00   
2    9474       10013      F  2038-09-03 00:00:00  2125-10-07 00:00:00   
3    9478       10017      F  2075-09-21 00:00:00  2152-09-12 00:00:00   
4    9479       10019      M  2114-06-20 00:00:00  2163-05-15 00:00:00   

              dod_hosp              dod_ssn  expire_flag  
0  2165-08-12 00:00:00  2165-08-12 00:00:00            1  
1  2126-08-28 00:00:00                 None            1  
2  2125-10-07 00:00:00  2125-10-07 00:00:00            1  
3                 None  2152-09-12 00:00:00            1  
4  2163-05-15 00:00:00  2163-05-15 00:00:00            1  

Contents of table 'PROCEDURES_ICD':
   row_id  subject_id  hadm_id  seq_num  icd9_code
0    3994       10114   167957        1       3605
1    3995       10114   167957 

## Initializing agent

In [ ]:
from langchain.agents import AgentType, create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.utilities import SQLDatabase

In [ ]:
db = SQLDatabase(engine)

### Initializing LLM

In [ ]:
from getpass import getpass
import os
os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key:")

OpenAI API Key:··········


In [ ]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(
    model_name="gpt-4",
    temperature=0
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:
from langchain.memory import ConversationSummaryBufferMemory

memory = ConversationSummaryBufferMemory(memory_key="chat_history",llm=llm)
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [ ]:
custom_prefix = """
If something is not present in a table check other tables. In case of count of patients always count unique patients.

Tables are linked by identifiers which usually have the suffix ‘ID’. For example, SUBJECT_ID refers to a unique patient, HADM_ID refers to a unique admission to the hospital, and ICUSTAY_ID refers to a unique admission to an intensive care unit.

Charted events such as notes, laboratory tests, and fluid balance are stored in a series of ‘events’ tables. For example the OUTPUTEVENTS table contains all measurements related to output for a given patient, while the LABEVENTS table contains laboratory test results for a patient.

Tables prefixed with ‘D_’ are dictionary tables and provide definitions for identifiers. For example, every row of CHARTEVENTS is associated with a single ITEMID which represents the concept measured, but it does not contain the actual name of the measurement. By joining CHARTEVENTS and D_ITEMS on ITEMID, it is possible to identify the concept represented by a given ITEMID.

Broadly speaking, five tables are used to define and track patient stays: ADMISSIONS; PATIENTS; ICUSTAYS; SERVICES; and TRANSFERS. Another five tables are dictionaries for cross-referencing codes against their respective definitions: D_CPT; D_ICD_DIAGNOSES; D_ICD_PROCEDURES; D_ITEMS; and D_LABITEMS. The remaining tables contain data associated with patient care, such as physiological measurements, caregiver observations, and billing information.

In some cases it would be possible to merge tables—for example, the D_ICD_PROCEDURES and CPTEVENTS tables both contain detail relating to procedures and could be combined—but our approach is to keep the tables independent for clarity, since the data sources are significantly different. Rather than combining the tables within MIMIC data model, we suggest researchers develop database views and transforms as appropriate.

Make sure to use tables prefixed with ‘D_’ as much as possible.
"""

agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    prefix=custom_prefix,

)

In [ ]:
agent.run("how many total patients are there")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with ``


ADMISSIONS, CALLOUT, CAREGIVERS, CHARTEVENTS, CPTEVENTS, DATETIMEEVENTS, DIAGNOSES_ICD, DRGCODES, D_CPT, D_ICD_DIAGNOSES, D_ICD_PROCEDURES, D_ITEMS, D_LABITEMS, ICUSTAYS, INPUTEVENTS_CV, INPUTEVENTS_MV, LABEVENTS, MICROBIOLOGYEVENTS, NOTEEVENTS, OUTPUTEVENTS, PATIENTS, PRESCRIPTIONS, PROCEDUREEVENTS_MV, PROCEDURES_ICD, SERVICES, TRANSFERS
Invoking: `sql_db_schema` with `PATIENTS`



CREATE TABLE "PATIENTS" (
	row_id BIGINT, 
	subject_id BIGINT, 
	gender TEXT, 
	dob TEXT, 
	dod TEXT, 
	dod_hosp TEXT, 
	dod_ssn TEXT, 
	expire_flag BIGINT
)

/*
3 rows from PATIENTS table:
row_id	subject_id	gender	dob	dod	dod_hosp	dod_ssn	expire_flag
9467	10006	F	2094-03-05 00:00:00	2165-08-12 00:00:00	2165-08-12 00:00:00	2165-08-12 00:00:00	1
9472	10011	F	2090-06-05 00:00:00	2126-08-28 00:00:00	2126-08-28 00:00:00	None	1
9474	10013	F	2038-09-03 00:00:00	2125-10-07 00:00:00	2125-10-07 00:00:00	2125-10-07 00:00:00	1
*/
Invokin

'There are 100 total patients.'

In [ ]:
agent.run("Are there any patients who had heart disease diagnosis more than 2 times give their names")



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with ``


ADMISSIONS, CALLOUT, CAREGIVERS, CHARTEVENTS, CPTEVENTS, DATETIMEEVENTS, DIAGNOSES_ICD, DRGCODES, D_CPT, D_ICD_DIAGNOSES, D_ICD_PROCEDURES, D_ITEMS, D_LABITEMS, ICUSTAYS, INPUTEVENTS_CV, INPUTEVENTS_MV, LABEVENTS, MICROBIOLOGYEVENTS, NOTEEVENTS, OUTPUTEVENTS, PATIENTS, PRESCRIPTIONS, PROCEDUREEVENTS_MV, PROCEDURES_ICD, SERVICES, TRANSFERS
Invoking: `sql_db_schema` with `DIAGNOSES_ICD, D_ICD_DIAGNOSES, PATIENTS`
responded: The tables that seem most relevant to this query are DIAGNOSES_ICD, D_ICD_DIAGNOSES, and PATIENTS. I will query the schema of these tables to understand their structure.


CREATE TABLE "DIAGNOSES_ICD" (
	row_id BIGINT, 
	subject_id BIGINT, 
	hadm_id BIGINT, 
	seq_num BIGINT, 
	icd9_code TEXT
)

/*
3 rows from DIAGNOSES_ICD table:
row_id	subject_id	hadm_id	seq_num	icd9_code
112344	10006	142345	1	99591
112345	10006	142345	2	99662
112346	10006	142345	3	5672
*/


CREATE TABLE "D_ICD_DIAGNOSE

'There are 11 patients who had a heart disease diagnosis more than 2 times. Their subject IDs are: 10088, 10124, 40204, 40456, 40503, 40601, 41795, 42033, 42302, 43735, and 43827.\n\nNow, I will query the PATIENTS table to get the names of these patients. However, please note that due to privacy regulations, the MIMIC-III database does not include patient names. Instead, each patient is identified by a unique SUBJECT_ID. Therefore, I can only provide the SUBJECT_IDs.'

In [ ]:
agent.run("Give all the past diagnosis of 10013")



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with ``


ADMISSIONS, CALLOUT, CAREGIVERS, CHARTEVENTS, CPTEVENTS, DATETIMEEVENTS, DIAGNOSES_ICD, DRGCODES, D_CPT, D_ICD_DIAGNOSES, D_ICD_PROCEDURES, D_ITEMS, D_LABITEMS, ICUSTAYS, INPUTEVENTS_CV, INPUTEVENTS_MV, LABEVENTS, MICROBIOLOGYEVENTS, NOTEEVENTS, OUTPUTEVENTS, PATIENTS, PRESCRIPTIONS, PROCEDUREEVENTS_MV, PROCEDURES_ICD, SERVICES, TRANSFERS
Invoking: `sql_db_schema` with `DIAGNOSES_ICD`
responded: The DIAGNOSES_ICD table seems to be the most relevant for past diagnoses. I will query the schema of this table.


CREATE TABLE "DIAGNOSES_ICD" (
	row_id BIGINT, 
	subject_id BIGINT, 
	hadm_id BIGINT, 
	seq_num BIGINT, 
	icd9_code TEXT
)

/*
3 rows from DIAGNOSES_ICD table:
row_id	subject_id	hadm_id	seq_num	icd9_code
112344	10006	142345	1	99591
112345	10006	142345	2	99662
112346	10006	142345	3	5672
*/
Invoking: `sql_db_schema` with `D_ICD_DIAGNOSES`
responded: The DIAGNOSES_ICD table contains the ICD9 codes for di

'The past diagnoses for the patient with the subject_id 10013 are:\n\n1. Unspecified septicemia\n2. Subendocardial infarction, initial episode of care\n3. Cardiogenic shock\n4. Pneumonia, organism unspecified\n5. Atrial fibrillation\n6. Other malignant lymphomas, unspecified site, extranodal and solid organ sites\n7. Mitral valve disorders\n8. Other and unspecified hyperlipidemia'

In [ ]:
agent.run("Can you identify patients with sepsis")



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with ``


ADMISSIONS, CALLOUT, CAREGIVERS, CHARTEVENTS, CPTEVENTS, DATETIMEEVENTS, DIAGNOSES_ICD, DRGCODES, D_CPT, D_ICD_DIAGNOSES, D_ICD_PROCEDURES, D_ITEMS, D_LABITEMS, ICUSTAYS, INPUTEVENTS_CV, INPUTEVENTS_MV, LABEVENTS, MICROBIOLOGYEVENTS, NOTEEVENTS, OUTPUTEVENTS, PATIENTS, PRESCRIPTIONS, PROCEDUREEVENTS_MV, PROCEDURES_ICD, SERVICES, TRANSFERS
Invoking: `sql_db_schema` with `DIAGNOSES_ICD, D_ICD_DIAGNOSES, ADMISSIONS`
responded: The most relevant tables for identifying patients with sepsis are likely to be DIAGNOSES_ICD, D_ICD_DIAGNOSES, and ADMISSIONS. Let's query the schema of these tables to understand their structure.


CREATE TABLE "ADMISSIONS" (
	row_id BIGINT, 
	subject_id BIGINT, 
	hadm_id BIGINT, 
	admittime TEXT, 
	dischtime TEXT, 
	deathtime TEXT, 
	admission_type TEXT, 
	admission_location TEXT, 
	discharge_location TEXT, 
	insurance TEXT, 
	language TEXT, 
	religion TEXT, 
	marital_status TEXT, 
	

"Here are some patients with sepsis:\n\n1. Patient with SUBJECT_ID 10006, HADM_ID 142345, admitted on '2164-10-23 21:09:00' and discharged on '2164-11-01 17:15:00'. Diagnosis: SEPSIS\n2. Patient with SUBJECT_ID 10038, HADM_ID 111115, admitted on '2144-02-09 17:53:00' and discharged on '2144-02-21 13:30:00'. Diagnosis: FAILURE TO THRIVE\n3. Patient with SUBJECT_ID 10088, HADM_ID 149044, admitted on '2107-05-12 18:00:00' and discharged on '2107-05-18 13:30:00'. Diagnosis: UROSEPSIS\n4. Patient with SUBJECT_ID 10124, HADM_ID 170883, admitted on '2192-04-16 20:57:00' and discharged on '2192-05-15 19:28:00'. Diagnosis: CONGESTIVE HEART FAILURE\n5. Patient with SUBJECT_ID 10132, HADM_ID 197611, admitted on '2123-08-23 20:00:00' and discharged on '2123-09-17 14:00:00'. Diagnosis: NON SMALL CELL CANCER;HYPOXIA\n\nPlease note that the diagnosis field in the ADMISSIONS table may not always indicate sepsis, even if the patient was diagnosed with sepsis according to the ICD9 code in the DIAGNOSES_